In [7]:
!pip3 uninstall webdriver_manager


Found existing installation: webdriver-manager 4.0.1
Uninstalling webdriver-manager-4.0.1:
  Would remove:
    /Users/arifrahman/Project/scraper/JobMiner/.venv/lib/python3.12/site-packages/webdriver_manager-4.0.1.dist-info/*
    /Users/arifrahman/Project/scraper/JobMiner/.venv/lib/python3.12/site-packages/webdriver_manager/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


In [1]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from pymongo import MongoClient
from dotenv import load_dotenv
import time

In [ ]:
# Load environment variables from .env file
load_dotenv()
MONGO_DB_URL = os.getenv("MONGO_DB_URL")

In [4]:
# Setup Chrome options
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Ensure GUI is off
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")


In [2]:
# Set path to chromedriver as per your configuration
webdriver_service = Service(ChromeDriverManager().install())



In [15]:
# Choose Chrome Browser
driver = webdriver.Chrome(service=webdriver_service, options=chrome_options)



In [16]:
# URL of the job listings page
url = 'https://www.indeed.com/jobs?q=data+engineer&l=California'
driver.get(url)



In [ ]:
# Connect to MongoDB
client = MongoClient(MONGO_DB_URL)
db = client['job_miner']
collection = db['jobs_link']

In [ ]:
# Find all job postings
job_postings = driver.find_elements(By.CLASS_NAME, 'job_seen_beacon')

print(job_postings)

In [ ]:
# Extract information from each job posting
for job in job_postings:
    # Extract job title
    title_elem = job.find_element(By.CLASS_NAME, 'jobTitle')
    title = title_elem.text.strip() if title_elem else 'N/A'
    
    # Extract company name    
    company_elem = job.find_element(By.CSS_SELECTOR, "span[data-testid='company-name']")
    company = company_elem.text.strip() if company_elem else 'N/A'
    
    # Extract job location
    location_elem = job.find_element(By.CSS_SELECTOR, "div[data-testid='text-location']")
    location = location_elem.text.strip() if location_elem else 'N/A'
    
    # Extract job link
    joblink_elem = job.find_element(By.CLASS_NAME, 'jobTitle').find_element(By.CLASS_NAME, 'jcs-JobTitle').get_attribute('href')
    joblink = joblink_elem.strip() if joblink_elem else 'N/A'
    
    # Print job information
    print(f'Job Title: {title}')
    print(f'Company: {company}')
    print(f'Location: {location}')
    print(f'Job Link: {joblink_elem}')
    print('-' * 40)

    # Create a dictionary to store the job data
    job_data = {
        'title': title,
        'company': company,
        'location': location,
        'job_link': joblink
    }
    
    # Insert the job data into MongoDB
    collection.insert_one(job_data)


In [14]:
# Close the browser
driver.quit()


In [ ]:
# Define the base URL and number of pages to scrape
base_url = 'https://www.indeed.com/jobs?q=data+engineer&l=California'
num_pages = 10

for page in range(num_pages):
    url = f'{base_url}&start={page * 10}'
    driver.get(url)
    time.sleep(10)  # wait for the page to load

    # Find all job postings
    job_postings = driver.find_elements(By.CLASS_NAME, 'job_seen_beacon')

    # Extract information from each job posting
    for job in job_postings:
        # Extract job title
        title_elem = job.find_element(By.CLASS_NAME, 'jobTitle')
        title = title_elem.text.strip() if title_elem else 'N/A'
        
        # Extract company name    
        company_elem = job.find_element(By.CSS_SELECTOR, "span[data-testid='company-name']")
        company = company_elem.text.strip() if company_elem else 'N/A'
        
        # Extract job location
        location_elem = job.find_element(By.CSS_SELECTOR, "div[data-testid='text-location']")
        location = location_elem.text.strip() if location_elem else 'N/A'
        
        # Extract job link
        joblink_elem = job.find_element(By.CLASS_NAME, 'jobTitle').find_element(By.CLASS_NAME, 'jcs-JobTitle').get_attribute('href')
        joblink = joblink_elem.strip() if joblink_elem else 'N/A'
        
        # Print job information
        print(f'Job Title: {title}')
        print(f'Company: {company}')
        print(f'Location: {location}')
        print(f'Job Link: {joblink}')
        print('-' * 40)

        # Create a dictionary to store the job data
        job_data = {
            'title': title,
            'company': company,
            'location': location,
            'job_link': joblink
        }
        
        # Insert the job data into MongoDB
        collection.insert_one(job_data)

In [38]:
base_url = 'https://www.indeed.com/jobs?q=data+engineer&l=California'
num_pages = 3

for page in range(num_pages):
    url = f'{base_url}&start={page * 10}'
    driver.get(url)
    time.sleep(3)  # wait for the page to load

    # Find all job postings
    job_postings = driver.find_elements(By.CLASS_NAME, 'job_seen_beacon')

    # Extract information from each job posting
    for job in job_postings:
        try:
            # Click the job to view details
            job.click()
            time.sleep(2)  # wait for the details panel to load

            # Extract job title
            title_elem = job.find_element(By.CLASS_NAME, 'jobTitle')
            title = title_elem.text.strip() if title_elem else 'N/A'
            
            # Extract company name    
            company_elem = job.find_element(By.CSS_SELECTOR, "span[data-testid='company-name']")
            company = company_elem.text.strip() if company_elem else 'N/A'
            
            # Extract job location
            location_elem = job.find_element(By.CSS_SELECTOR, "div[data-testid='text-location']")
            location = location_elem.text.strip() if location_elem else 'N/A'
            
            # Extract job link
            joblink_elem = job.find_element(By.CLASS_NAME, 'jobTitle').find_element(By.CLASS_NAME, 'jcs-JobTitle').get_attribute('href')
            joblink = joblink_elem.strip() if joblink_elem else 'N/A'

            # Extract job details (e.g., salary, employment type, etc.)
            detail_sections = driver.find_elements(By.CLASS_NAME, 'jobsearch-JobDescriptionSection-sectionItem')
            details = {}
            for section in detail_sections:
                key_elem = section.find_element(By.CLASS_NAME, 'jobsearch-JobDescriptionSection-sectionItemKey')
                value_elem = section.find_element(By.CLASS_NAME, 'jobsearch-JobDescriptionSection-sectionItemValue')
                key = key_elem.text.strip() if key_elem else 'N/A'
                value = value_elem.text.strip() if value_elem else 'N/A'
                details[key] = value

            # Extract job description
            description_elem = driver.find_element(By.CLASS_NAME, 'jobsearch-JobComponent-description')
            description = description_elem.text.strip() if description_elem else 'N/A'
            
            # Print job information
            print(f'Job Title: {title}')
            print(f'Company: {company}')
            print(f'Location: {location}')
            print(f'Job Link: {joblink}')
            print(f'Details: {details}')
            print(f'Description: {description}')
            print('-' * 40)

            # Create a dictionary to store the job data
            job_data = {
                'title': title,
                'company': company,
                'location': location,
                'job_link': joblink,
                'details': details,
                'description': description
            }
            
            # Insert the job data into MongoDB
            collection.insert_one(job_data)
        except Exception as e:
            print(f"Error processing job: {e}")
            continue

Job Title: Data Engineer -
Company: Kandji
Location: San Francisco, CA
Job Link: https://www.indeed.com/rc/clk?jk=4b8606224f6c913b&bb=ZIMRsFk2_AHPe4UietgcKB9eKSvgtq7xxccsGfhvFPotIV0PpOazgMBKfrvmDZWK04gjK0gZesaJnN0eNcRlqBvEKXQcAmks6POrMt2CEkMUuKxbtZxD1VjUbyibHtKq&xkcb=SoCm67M3AbjOLl2M2p0LbzkdCdPP&fccid=e7e14440c198dd78&vjs=3
Details: {}
Description: Profile insights
Find out how your skills align with the job description
Skills
Do you have experience in SaaS?
Yes
No
Job details
Here’s how the job details align with your profile
.
Pay
$120,000 - $130,000 a year
Job type
Full-time
Location
San Francisco, CA
Benefits
Pulled from the full job description
401(k) 4% Match
401(k) matching
Dental insurance
Gym membership
Health insurance
Paid parental leave
Paid time off
Show more
Full job description
About Kandji

Kandji is the Apple Device Management and Security Platform. Kandji empowers companies to manage and secure Apple devices in the enterprise and at scale. By centrally securing and ma

In [20]:
driver.find_element(By.CSS_SELECTOR, 'div[data-testid="jobsearch-OtherJobDetailsContainer"]').find_element(By.CSS_SELECTOR, 'div[id="salaryInfoAndJobType"]').text.strip()

'$78,700 - $145,100 a year - Full-time'